This notebook shows how to run playoff scenarios prior to the end of regular season,
and identify how each team might win their division, whether they "control their
own destiny" etc. This example is scoped to the last two weeks of regular
season (weeks 17 and 18). If you're running the analysis after the end of
the season then these scores will be in the database, but you can clear them
and replace them with your own assumptions as shown below.

In the future I hope to do similar notebooks for wilcard slots and home-field advantage

In [4]:
from nfl import NFL
import pandas as pd
import numpy as np
import time

# for progress bars
from ipywidgets import IntProgress
from IPython.display import display

nfl = NFL().update()

In [5]:
# For reference, print conference standings not counting weeks 17 and 18
weeks = [17, 18]
nfl.clear(weeks)
nfl('NFC')

NFC
                       name        div overall                    division  \
                                           win loss tie       pct      win   
team                                                                         
PHI     Philadelphia Eagles   NFC-East      12    2   0  0.857143        3   
WSH   Washington Commanders   NFC-East       9    5   0  0.642857        2   
DAL          Dallas Cowboys   NFC-East       6    8   0  0.428571        3   
NYG         New York Giants   NFC-East       2   12   0  0.142857        0   
DET           Detroit Lions  NFC-North      12    2   0  0.857143        4   
MIN       Minnesota Vikings  NFC-North      12    2   0  0.857143        3   
GB        Green Bay Packers  NFC-North      10    4   0  0.714286        1   
CHI           Chicago Bears  NFC-North       4   10   0  0.285714        0   
TB     Tampa Bay Buccaneers  NFC-South       8    6   0  0.571429        2   
ATL         Atlanta Falcons  NFC-South       7    7   0       0.5        4   
NO       New Orleans Saints  NFC-South       5    9   0  0.357143        2   
CAR       Carolina Panthers  NFC-South       3   11   0  0.214286        1   
LAR        Los Angeles Rams   NFC-West       8    6   0  0.571429        3   
SEA        Seattle Seahawks   NFC-West       8    6   0  0.571429        3   
ARI       Arizona Cardinals   NFC-West       7    7   0       0.5        2   
SF      San Francisco 49ers   NFC-West       6    8   0  0.428571        1   

                    conference                     
     loss tie   pct        win loss tie       pct  
team                                               
PHI     0   0   1.0          7    2   0  0.777778  
WSH     2   0   0.5          6    3   0  0.666667  
DAL     1   0  0.75          4    5   0  0.444444  
NYG     5   0   0.0          1    9   0       0.1  
DET     0   0   1.0          8    1   0  0.888889  
MIN     1   0  0.75          7    2   0  0.777778  
GB      3   0  0.25          5    4   0  0.555556  
CHI     4   0   0.0          2    7   0  0.222222  
TB      2   0   0.5          6    3   0  0.666667  
ATL     1   0   0.8          6    3   0  0.666667  
NO      3   0   0.4          4    6   0       0.4  
CAR     3   0  0.25          2    7   0  0.222222  
LAR     1   0  0.75          5    5   0       0.5  
SEA     2   0   0.6          4    5   0  0.444444  
ARI     2   0   0.5          3    6   0  0.333333  
SF      4   0   0.2          4    6   0       0.4

In [6]:
# Specify the division and week range to run
div = 'NFC-South'
weeks = [17, 18]

nfl.clear(weeks)
nfl(div)

NFC-South
                      name        div overall                    division  \
                                          win loss tie       pct      win   
team                                                                        
TB    Tampa Bay Buccaneers  NFC-South       8    6   0  0.571429        2   
ATL        Atlanta Falcons  NFC-South       7    7   0       0.5        4   
NO      New Orleans Saints  NFC-South       5    9   0  0.357143        2   
CAR      Carolina Panthers  NFC-South       3   11   0  0.214286        1   

                    conference                     
     loss tie   pct        win loss tie       pct  
team                                               
TB      2   0   0.5          6    3   0  0.666667  
ATL     1   0   0.8          6    3   0  0.666667  
NO      3   0   0.4          4    6   0       0.4  
CAR     3   0  0.25          2    7   0  0.222222

In [7]:
# limit scope to teams still in contention
st = nfl(div).standings
teams = set(st[st[('overall','win')] >= st.iloc[0][('overall','win')] - len(weeks)].index)

# print the relevant schedule for reference. It's helpful to know
# who plays who
nfl.schedule(teams, weeks, by='game')

ateam  hscore  ascore         date
week hteam                                   
17   TB      CAR     NaN     NaN  12/29 13:00
     WSH     ATL     NaN     NaN  12/29 20:20
18   ATL     CAR     NaN     NaN    1/5 00:00
     TB       NO     NaN     NaN    1/5 00:00

In [8]:
# iterate over all possible outcomes and count the number
# of times each team wins the division.
# this can take 30 seconds or more in fast mode for a 2-week span

results = pd.DataFrame(columns=pd.MultiIndex.from_product([weeks, teams], names=['week','team']))
results.index.name = 'scenario'
results[('result','outcome')] = np.nan
results[('result','rule')] = np.nan

# calculate the number of scenarios to set up the progress bar
# it's the number of potential outcomes (win,loss.tie = 3)
# to the power of the number of relevant games (4 in this case)
# pow(3, 4) = 81

bar = IntProgress(min=0, max=3 ** len(nfl.schedule(teams, weeks, by='game')))
display(bar)

start = time.time()
for elem in nfl.scenarios(weeks, teams):
    nfl.clear(weeks)
    nfl.set(elem)

    sch = nfl.schedule(teams, weeks, by='team')

# NB: if you are only interested in specific scenarios, e.g.
# ones where a particuar team wins all their remaining games
# you can typically save a lot of time by querying the resulting
# schedule for the current scenario at this point and avoid
# the call to tiebreaks, which even in fast mode can be
# very time consuming
    
    t = nfl.tiebreaks(teams, fast=True)

    z = len(results)
    results.loc[z] = sch['wlt']
    results.loc[z, ('result','outcome')] = t.index[0]
    if len(t) > 1:
        results.loc[z, ('result','rule')] = t.iloc[1]
    else:
        results.loc[z, ('result','rule')] = 'overall'

    bar.value += 1

bar.layout.display = 'none'
print('Elapsed time: {}'.format(time.time() - start))
results

IntProgress(value=0, max=81)

Elapsed time: 9.601961851119995


week        17         18        result         
team       ATL   TB   ATL    TB outcome     rule
scenario                                        
0         loss  win   win   win      TB  overall
1         loss  win   win  loss      TB  overall
2         loss  win   win   tie      TB  overall
3         loss  win  loss   win      TB  overall
4         loss  win  loss  loss      TB  overall
...        ...  ...   ...   ...     ...      ...
76         tie  tie  loss  loss      TB  overall
77         tie  tie  loss   tie      TB  overall
78         tie  tie   tie   win      TB  overall
79         tie  tie   tie  loss      TB  overall
80         tie  tie   tie   tie      TB  overall

[81 rows x 6 columns]

In [9]:
# This shows how many scenarios result in each team winning the division
results.groupby(('result','outcome')).count()

week               17      18     result
team              ATL  TB ATL  TB   rule
(result, outcome)                       
ATL                15  15  15  15     15
TB                 66  66  66  66     66

In [26]:
# this will report all scenarios where the specified team wins
# their remaining games, answering the question whether they
# "control their own destiny." If so then the result will invariably
# be that the team wins the division, as shown in the 2nd column from the right

# Note that if a rule shows as conference|overall-rank or any form of netpoints,
# the analysis is not necessarily valid because the scenarios are based on
# outcomes only without specifying point spreads

test_team = st.index[1]

results[(results.xs(test_team,level=1,axis=1) == 'win').all(axis=1)]

week       17         18        result              
team      ATL    TB  ATL    TB outcome          rule
scenario                                            
9         win   win  win   win      TB       overall
10        win   win  win  loss     ATL  head-to-head
11        win   win  win   tie      TB       overall
36        win  loss  win   win     ATL  head-to-head
37        win  loss  win  loss     ATL       overall
38        win  loss  win   tie     ATL       overall
63        win   tie  win   win      TB       overall
64        win   tie  win  loss     ATL       overall
65        win   tie  win   tie     ATL  head-to-head

In [15]:
# Are there any situations where ATL wins only one game and still wins the division?
results[results[('result','outcome')]==test_team]

week        17          18        result              
team       ATL    TB   ATL    TB outcome          rule
scenario                                              
10         win   win   win  loss     ATL  head-to-head
28        loss  loss   win  loss     ATL  head-to-head
36         win  loss   win   win     ATL  head-to-head
37         win  loss   win  loss     ATL       overall
38         win  loss   win   tie     ATL       overall
40         win  loss  loss  loss     ATL  head-to-head
43         win  loss   tie  loss     ATL       overall
44         win  loss   tie   tie     ATL  head-to-head
46         tie  loss   win  loss     ATL       overall
47         tie  loss   win   tie     ATL  head-to-head
52         tie  loss   tie  loss     ATL  head-to-head
64         win   tie   win  loss     ATL       overall
65         win   tie   win   tie     ATL  head-to-head
70         win   tie   tie  loss     ATL  head-to-head
73         tie   tie   win  loss     ATL  head-to-head

In [27]:
# are there any scenarios where ATL wins only 1 game and still takes the division?
results[(results.xs(test_team,level=1,axis=1)=='win').sum(axis=1)==1]

week        17          18        result              
team       ATL    TB   ATL    TB outcome          rule
scenario                                              
0         loss   win   win   win      TB       overall
1         loss   win   win  loss      TB       overall
2         loss   win   win   tie      TB       overall
12         win   win  loss   win      TB       overall
13         win   win  loss  loss      TB       overall
14         win   win  loss   tie      TB       overall
15         win   win   tie   win      TB       overall
16         win   win   tie  loss      TB       overall
17         win   win   tie   tie      TB       overall
18         tie   win   win   win      TB       overall
19         tie   win   win  loss      TB       overall
20         tie   win   win   tie      TB       overall
27        loss  loss   win   win      TB       overall
28        loss  loss   win  loss     ATL  head-to-head
29        loss  loss   win   tie      TB       overall
39         win  loss  loss   win      TB       overall
40         win  loss  loss  loss     ATL  head-to-head
41         win  loss  loss   tie      TB       overall
42         win  loss   tie   win      TB       overall
43         win  loss   tie  loss     ATL       overall
44         win  loss   tie   tie     ATL  head-to-head
45         tie  loss   win   win      TB       overall
46         tie  loss   win  loss     ATL       overall
47         tie  loss   win   tie     ATL  head-to-head
54        loss   tie   win   win      TB       overall
55        loss   tie   win  loss      TB       overall
56        loss   tie   win   tie      TB       overall
66         win   tie  loss   win      TB       overall
67         win   tie  loss  loss      TB       overall
68         win   tie  loss   tie      TB       overall
69         win   tie   tie   win      TB       overall
70         win   tie   tie  loss     ATL  head-to-head
71         win   tie   tie   tie      TB       overall
72         tie   tie   win   win      TB       overall
73         tie   tie   win  loss     ATL  head-to-head
74         tie   tie   win   tie      TB       overall